# 3. Feature Engineering

Merges datasets and builds required features.


In [ ]:

import pandas as pd
from pathlib import Path
from src.features import add_ema, add_derived_features

DATA_DIR = Path("../data")

spot = pd.read_csv(DATA_DIR/"nifty_spot_5min.csv", parse_dates=["timestamp"])
fut  = pd.read_csv(DATA_DIR/"nifty_futures_5min.csv", parse_dates=["timestamp"])
opt  = pd.read_csv(DATA_DIR/"nifty_options_5min.csv", parse_dates=["timestamp"])

merged = spot.merge(fut, on="timestamp", suffixes=("_spot","_fut"))
merged = merged.merge(opt.drop(columns=["strike"], errors="ignore"), on="timestamp", how="left")

merged = merged.rename(columns={"close_spot":"spot_close","close_fut":"fut_close"})
df = merged.copy()

# Ensure greek columns exist (real pipeline should compute Greeks via src/greeks.py)
for col in ["call_delta","put_delta","call_gamma","call_vega"]:
    if col not in df.columns:
        df[col] = 0.0

df = add_ema(df)
df = add_derived_features(df)

merged.to_csv(DATA_DIR/"nifty_merged_5min.csv", index=False)
df.to_csv(DATA_DIR/"nifty_features_5min.csv", index=False)
print("Saved merged + features CSVs.")
df.head()
